# 07 Functions as first class objects
Some notes, observations and questions along chapter 07.

### Definition: first class objects
- can be:
    - created at runtime
    - assigned to variable names
    - passed as an argument in a function
    - returned in a function

### Treating a function like an object

Functions are normal objects:

In [5]:
# we can define it
def factorial(n):
    """returns n!"""
    return 1 if n < 2 else n * factorial(n - 1)

In [6]:
# we can call it
factorial(42)

1405006117752879898543142606244511569936384000000000

In [9]:
# they have a __doc__ attribute
factorial.__doc__

'returns n!'

In [8]:
# they have a type
type(factorial)

function

In [11]:
# we can assign functions a variable name
fact = factorial
fact

<function __main__.factorial(n)>

In [12]:
# we can call the function by that variable name
fact(5)

120

In [13]:
# we can pass it into a map() function
map(factorial, range(11))

In [14]:
# and use it in map()
list(map(factorial, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

Calling map(function, iterable) returns an iterable where each item is the result of calling the first argument (a function) to successive elements of the second argument (an iterable).

### Higher-order functions
- function that takes a function as an argument or returns a function as the result
- for instance `map()`, `filter()`, `reduce()`
- but they are not as useful anymore since the introduction of list comprehensions and generator expressions, which are more readable

In [15]:
# comparison, both the same output
list(map(factorial, range(6)))
[factorial(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [16]:
# comparison, both the same output
list(map(factorial, filter(lambda n: n % 2, range(6)))) 
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

### Anonymous Functions
"The best use of anonymous functions is in the context of an argument list for a higher-order function."

In [20]:
# returning sorted words, but sorted by last letter first
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### The Nine Flavors of Callable Objects
- callable with `()` are: 
    - functions and methods
    - classes that define `__call__`
    - functions or methods that use the `yield` keyword in their body; when called, they return a generator object
    - and more

### User-Defined Callable Types


In [22]:
# shortcut to bingo.pick(): bingo()
import random

class BingoCage:

    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        return self.pick()

I am always unsure why we need such a shortcut, if it complicates the code and hides the internal mechanisms from the user...

In [30]:
bingo = BingoCage(range(3))
bingo.pick()

1

In [31]:
bingo()

2

Answer to my concern above: "A class implementing `__call__` is an easy way to create function-like objects that have some internal state that must be kept across invocations, like the remaining items in the BingoCage."

And callable objects are also used in decorators sometimes, when they need to remember something between the calls (for instance caching).

### From Positional to Keyword-Only Parameters

In [32]:
# usage of * and **
def tag(name, *content, class_=None, **attrs):
    """Generate one or more HTML tags"""
    if class_ is not None:
        attrs['class'] = class_
    attr_pairs = (f' {attr}="{value}"' for attr, value in sorted(attrs.items()))
    attr_str = ''.join(attr_pairs)
    if content:
        elements = (f'<{name}{attr_str}>{c}</{name}>' for c in content)
        return '\n'.join(elements)
    else:
        return f'<{name}{attr_str} />'


In [33]:
tag('br')

'<br />'

In [34]:
tag('p', 'hello')

'<p>hello</p>'

In [36]:
print(tag('p', 'hello', 'world'))

<p>hello</p>
<p>world</p>


In [37]:
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [38]:
print(tag('p', 'hello', 'world', class_='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [39]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'class': 'framed'}
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

#### keyword-only arguments

All the params after `*` need to be keyword arguments when calling the function.

In [40]:
def f(a, *, b):
    return a, b

f(1, b=2)

(1, 2)

In [41]:
f(1, 2)

TypeError: f() takes 1 positional argument but 2 were given

#### positional-only arguments

All the params before `/` need to be positional only when calling the function.

In [42]:
def divmod(a, b, /):
    return (a // b, a % b)

divmod(5,2)

(2, 1)

In [43]:
divmod(a=5, b=2)

TypeError: divmod() got some positional-only arguments passed as keyword arguments: 'a, b'

### Packages for Functional Programming
#### The `operator` Module

`itemgetter` and `attrgetter` are factories that build custom functions to pick items from sequences or read attributes from objects.

In [45]:
# common use of itemgetter: sorting a list of tuples by the value of one field
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('São Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


`itemgetter()` supports sequences, mappings and any class that implements `__getitem__`.
It's most common use case is extracting a certain element from a data structure that contains several sub-elements.

`attrgetter()` creates functions to extract object attributes by name. It can also navigate through nested objects to get an attribute (if we pass object.attribute.attribute, for instance).

Like the former two, `methodcaller()` is a factory that creates a method on the fly:

In [9]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [10]:
hyphenate = methodcaller('replace', ' ', '-')
hyphenate(s)

'The-time-has-come'

### Freezing Arguments with functools.partial
Given a callable, `functools.partial()` produces a new callable with some of the arguments of the original callable bound to predetermined values. It takes a callable as first argument, followed by an arbitrary number of positional and keyword arguments to bind.

Usable more for user APIs, not so much when you use it for yourself.

And another [justification on functools.partial](https://stackoverflow.com/questions/3252228/python-why-is-functools-partial-necessary) compared to using a lambda function.

### Introspection of Function Parameters
That part had been moved from the book to here: https://www.fluentpython.com/extra/function-introspection/

- functions have a `__doc__` attribute to access their docstrings
- invoking `help(func)` on the Python console has the same effect
- `__dict__` stores user attributes assigned to it

Questions: The text suggests that functions have the same attributes as objects. Is calling a function the same as instantiating a class (so an object is created) in terms for available attributes?

These are attributes that don't exist in objects that inherit from the object class, but in functions:
`['__annotations__', '__call__', '__closure__', '__code__', '__defaults__',`
`'__get__', '__globals__', '__kwdefaults__', '__name__', '__qualname__']`

A description of their use can be found [here](https://www.fluentpython.com/extra/function-introspection/).

But `__defaults__` is complicated to use, since it mixes params from the signature and local variables.

"`inspect.signature` returns an `inspect.Signature` object, which has a parameters attribute that lets us read an ordered mapping of names to inspect.Parameter objects. Each Parameter instance has attributes such as name, default, and kind. The special value inspect._empty denotes parameters with no default, which makes sense considering that None is a valid—​and popular—​default value."